In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

In [3]:
f = open("temp.txt", "r")
lines = f.read().split("\n")
lines= [int(i) for i in lines]

In [4]:
lines[0], lines[-1]

(115, 65)

In [10]:
lines.extend([0,max(lines)+3])

In [13]:
df = pd.DataFrame(lines,columns=["num"])
df = df.sort_values(by=["num"])

In [15]:
new_df = df.diff()
new_df

,num
108,NaN
38,1.0
20,1.0
43,1.0
84,1.0
...,...
80,3.0
3,3.0
11,1.0
35,1.0


In [21]:
import collections
print(collections.Counter(new_df["num"].tolist()[1:]))

Counter({1.0: 69, 3.0: 40})


In [22]:
69*40

2760

In [45]:
li = "".join(new_df.mask(new_df["num"]==1,"0").mask(new_df["num"]==3,"1")["num"].tolist()[1:])
li

'0000100001000111001110001001001100001000100010000100001001001000110001001000111000100110110000110000111111001'

In [52]:
new_li = [len(i)-1 for i in li.split("1") if len(i)>1]
new_li

[3, 3, 2, 1, 2, 1, 1, 3, 2, 2, 3, 3, 1, 1, 2, 2, 1, 2, 2, 1, 3, 3, 1]

In [53]:
count = 1
for i in new_li:
    count *= min(7,2**i)

In [54]:
count

13816758796288

In [13]:
def check_if_find(li, t):
    for n in li:
        new_n = t - n
        if (new_n != n) and (new_n in li):
            return True
    return False

In [16]:
preamble = 25

for current in range(preamble,len(lines)):
    input_li = lines[current - 25 : current]
    target = lines[current]
    if not check_if_find(input_li,target):
        print(target)
        break

23278925


In [17]:
target = 23278925

In [18]:
total = len(lines)
for i in range(total):
    for j in range(i+1,total):
        new_li = lines[i:j+1]
        if sum(new_li)==target:
            print(min(new_li)+max(new_li))
            break

4011064


In [9]:
table = [(i.split(" ")[0], int(i.split(" ")[1])) for i in lines]

In [10]:
table[0]

('acc', -15)

In [11]:
count = 0
point = 0
done = []
while point not in done:
    done.append(point)
    act, num = table[point]
    if act == "acc":
        count += num
        point += 1
    elif act == "jmp":
        point += num
    else:
        point += 1
count

1200

In [13]:
def check_pass(table):
    count = 0
    point = 0
    done = []
    while point not in done:
        done.append(point)
        act, num = table[point]
        if act == "acc":
            count += num
            point += 1
        elif act == "jmp":
            point += num
        else:
            point += 1
        if point >= len(table):
            return True
    return False

In [14]:
check_pass(table)

False

In [17]:
for i in range(len(table)):
    new_table = table[:]
    if table[i][0] == "jmp":
        new_table[i] = ("nop", table[i][1])
    elif table[i][0] == "nop":
        new_table[i] = ("jmp", table[i][1])
    else:
        continue
    if check_pass(new_table):
        break

In [18]:
count = 0
point = 0
done = []
while point not in done:
    done.append(point)
    act, num = new_table[point]
    if act == "acc":
        count += num
        point += 1
    elif act == "jmp":
        point += num
    else:
        point += 1
count

IndexError: list index out of range

In [19]:
count

1023

In [5]:
df = pd.read_csv("temp.csv")
df

,num
0,FFBBFFFLRL
1,FFBBFBBRRL
2,FBBBFFBLRL
3,BBFBFFBLRR
4,BFBBBFFLLL
...,...
927,BBFBFFFLRL
928,FFBFFFFRLL
929,FFFBBBFRLL
930,FBBBBBFLLL


In [23]:
def find_row(s, l, h):
    s = s.replace(l, "0")
    s = s.replace(h, "1")
    return int(s, 2)


find_row("FBFBBFF", "F", "B")

44

In [30]:
find_row("LRL", "L", "R")

2

In [31]:
df["row"] = df.apply(lambda x: find_row(x["num"][:7], "F", "B"), axis=1)
df["col"] = df.apply(lambda x: find_row(x["num"][-3:], "L", "R"), axis=1)

In [33]:
df["id"] = df["row"] * 8 + df["col"]

In [39]:
df.id.max(), df.id.min()

(947, 15)

In [38]:
id_list = df["id"].tolist()

In [40]:
for id in range(15, 947, 1):
    if id not in id_list:
        print(id)

636


In [25]:
"FFBBFFFLRL"[-3:]

'LRL'

In [15]:
127 - int("1010011", 2)

44

In [16]:
int("0101100", 2)

44

In [19]:
int("101", 2)

5

In [6]:
lines[0], lines[-1]

('pale cyan bags contain 2 posh black bags, 4 wavy gold bags, 2 vibrant brown bags.',
 'pale magenta bags contain 1 clear gold bag, 5 posh fuchsia bags, 2 faded cyan bags.')

In [39]:
import re

In [63]:
def find_bags(s):
    pattern_first = "([a-z ]*) bags contain"
    matchObj = re.findall(pattern_first, s)
    first_bag = matchObj[0]
    # print(first_bag)
    pattern_rest = "(\d+) ([a-z]* [a-z]*) bag[s]?"
    matchObj = re.findall(pattern_rest, s)
    # print(matchObj)
    res_dict = {}
    for n, bag in matchObj:
        res_dict[bag] = int(n)
    return first_bag, res_dict

In [64]:
find_bags("faded olive bags contain 1 muted indigo bag.")

('faded olive', {'muted indigo': 1})

In [72]:
bag_dict = {}
for color, b_dict in map(find_bags, lines):
    bag_dict[color] = b_dict

In [85]:
can_list = ["shiny gold"]
while True:
    current_num = len(can_list)
    for color, b_dict in bag_dict.items():
        if color not in can_list:
            if len(np.intersect1d(can_list, list(b_dict.keys()))) > 0:
                can_list.append(color)
    new_num = len(can_list)
    print(new_num)
    if new_num == current_num:
        break

25
81
146
162
165
165


In [91]:
sum(bag_dict["shiny gold"].values())

14

In [93]:
count = 0
for c_d in bag_dict["shiny gold"]:
    count += sum(c_d.values())

AttributeError: 'str' object has no attribute 'values'

In [101]:
count_list = []


def find_inside_bags(c, n=1):
    new_count = sum(bag_dict[c].values())
    count_list.append(new_count * n)
    #     print(new_count)
    if new_count == 0:
        return
    else:
        for new_c, mul in bag_dict[c].items():
            new_n = n * mul
            find_inside_bags(new_c, new_n)

In [102]:
find_inside_bags("shiny gold")
count_list

In [103]:
sum(count_list)

7872

In [92]:
c_d

NameError: name 'c_d' is not defined

In [12]:
blocs = []
new_bloc = ""
for line in lines:
    #     line = line[:-1]
    if len(line) == 0:
        blocs.append(new_bloc)
        new_bloc = ""
        continue
    new_bloc += " " + line
blocs.append(new_bloc)
blocs = [b[1:] for b in blocs]

In [22]:
blocs[0]

'bahmtpujokxdqlyi efqksvrctzhmngw'

In [16]:
line = "a a a tv"
len(set([i for i in line if i != " "]))

3

In [25]:
b = "bahmtpujokxdqlyi efqksvrctzhmngw"
b.split(" ")

['bahmtpujokxdqlyi', 'efqksvrctzhmngw']

In [ ]:
list(set(list1).intersection(list2))

In [24]:
from functools import reduce

reduce(np.intersect1d, b.split(" "))

array([], dtype='<U16')

In [27]:
from functools import reduce

reduce(
    np.intersect1d, ([i for i in "bahmtpujokxdqlyi"], [i for i in "efqksvrctzhmngw"])
)

array(['h', 'k', 'm', 'q', 't'], dtype='<U1')

In [40]:
count_sum = 0
for b in blocs:
    new_count = len(set([i for i in b if i != " "]))
    count_sum += new_count
count_sum

6662

In [39]:
count_sum = 0
for b in blocs:
    new_count = len(reduce(np.intersect1d, ([j for j in i] for i in b.split(" "))))
    #     print(new_count)
    count_sum += new_count
count_sum

3382

In [59]:
user = []
for b in blocs:
    new_p = {}
    for i in b.split(" "):
        k, v = i.split(":")
        new_p[k] = v
    user.append(new_p)

In [62]:
def RepresentsInt(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [73]:
def check_hgt(s):
    h = s[:-2]
    if RepresentsInt(h):
        h = int(h)
        if s[-2:] == "cm":
            if (h >= 150) and (h <= 193):
                return True
        elif s[-2:] == "in":
            if (h >= 59) and (h <= 76):
                return True
    return False


def check_hcl(s):
    if s[0] == "#":
        s = s[1:]
        if len(s) == 6:
            return all(
                [
                    ((ord(l) >= 48) and (ord(l) <= 57))
                    or ((ord(l) >= 97) and (ord(l) <= 102))
                    for l in s
                ]
            )
    return False


def check_ecl(s):
    return s in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]


def check_pid(s):
    return (len(s) == 9) and RepresentsInt(s)

In [75]:
count = 0
for u in user:
    check1 = (
        ("byr" in u)
        and RepresentsInt(u["byr"])
        and (int(u["byr"]) >= 1920)
        and (int(u["byr"]) <= 2002)
    )
    check2 = (
        ("iyr" in u)
        and RepresentsInt(u["iyr"])
        and (int(u["iyr"]) >= 2010)
        and (int(u["iyr"]) <= 2020)
    )
    check3 = (
        ("eyr" in u)
        and RepresentsInt(u["eyr"])
        and (int(u["eyr"]) >= 2020)
        and (int(u["eyr"]) <= 2030)
    )
    check4 = ("hgt" in u) and check_hgt(u["hgt"])
    check5 = ("hcl" in u) and check_hcl(u["hcl"])
    check6 = ("ecl" in u) and check_ecl(u["ecl"])
    check7 = ("pid" in u) and check_pid(u["pid"])
    check8 = "cid" in u
    if all([check1, check2, check3, check4, check5, check6, check7]):
        count += 1
#         print(u)
count

175

In [ ]:
user

In [ ]:
user

In [9]:
lines[1]

'.#..#...##..#....##........##..'

In [11]:
col = len(lines[0])
row = len(lines)
col, row

(31, 323)

In [16]:
right = 3
down = 1

In [17]:
count = 0
c = right
for r in range(1, row):
    s = lines[r][c % col]
    if s == "#":
        count += 1
    c += right
print(count)

216


In [25]:
def toto(right, down):
    count = 0
    c = right
    for r in range(down, row, down):
        s = lines[r][c % col]
        if s == "#":
            count += 1
        c += right
    return count

In [26]:
toto(1, 1), toto(3, 1), toto(5, 1), toto(7, 1), toto(1, 2)

(79, 216, 91, 96, 45)

In [27]:
toto(1, 1) * toto(3, 1) * toto(5, 1) * toto(7, 1) * toto(1, 2)

6708199680

In [20]:
toto(1, 1), toto(3, 1), toto(5, 1), toto(7, 1), toto(1, 2)

(79, 216, 91, 96, 46)

In [21]:
toto(1, 1) * toto(3, 1) * toto(5, 1) * toto(7, 1) * toto(1, 2)

6857270784

In [ ]:
for r in range(2, 323, 2):
    print(r)

In [9]:
df

,Min,Max,Letter,Code
0,15,19,k,kkkkkkkkkkkkzkkkkkkk
1,1,11,s,sbssswsqsssssrlss
2,8,9,b,pbbbbbbkbz
3,4,10,w,wwccwcqwdmbktjrxhw
4,1,6,x,jvscgqsnt
...,...,...,...,...
995,9,10,m,wmvmhmmmxddzmmmm
996,2,11,c,flcqrnrqmcccs
997,2,3,f,qfdx
998,11,15,k,kkkkdkkkkkgkkkkkkrq


In [10]:
df["Count"] = df.apply(lambda row: row["Code"].count(row["Letter"]), axis=1)
df

,Min,Max,Letter,Code,Count
0,15,19,k,kkkkkkkkkkkkzkkkkkkk,19
1,1,11,s,sbssswsqsssssrlss,12
2,8,9,b,pbbbbbbkbz,7
3,4,10,w,wwccwcqwdmbktjrxhw,5
4,1,6,x,jvscgqsnt,0
...,...,...,...,...,...
995,9,10,m,wmvmhmmmxddzmmmm,9
996,2,11,c,flcqrnrqmcccs,4
997,2,3,f,qfdx,1
998,11,15,k,kkkkdkkkkkgkkkkkkrq,15


In [12]:
((df["Count"] >= df["Min"]) & (df["Count"] <= df["Max"])).sum()

655

In [16]:
df["Test1"] = df.apply(lambda row: row["Code"][row["Min"] - 1] == row["Letter"], axis=1)
df["Test2"] = df.apply(lambda row: row["Code"][row["Max"] - 1] == row["Letter"], axis=1)
df

,Min,Max,Letter,Code,Count,Test1,Test2
0,15,19,k,kkkkkkkkkkkkzkkkkkkk,19,True,True
1,1,11,s,sbssswsqsssssrlss,12,True,True
2,8,9,b,pbbbbbbkbz,7,False,True
3,4,10,w,wwccwcqwdmbktjrxhw,5,False,False
4,1,6,x,jvscgqsnt,0,False,False
...,...,...,...,...,...,...,...
995,9,10,m,wmvmhmmmxddzmmmm,9,False,False
996,2,11,c,flcqrnrqmcccs,4,False,True
997,2,3,f,qfdx,1,True,False
998,11,15,k,kkkkdkkkkkgkkkkkkrq,15,False,True


In [20]:
((df["Test1"].astype("int") + df["Test2"].astype("int")) == 1).sum()

673

In [19]:
li = df["Num"].tolist()

In [14]:
for i in li:
    if (2020 - i) in li:
        print(i, 2020 - i)
        print(i * (2020 - i))

1847 173
319531
173 1847
319531


In [20]:
from itertools import combinations

for x, y, z in combinations(li, 3):
    if x + y + z == 2020:
        print(x, y, z)
        print(x * y * z)

873 795 352
244300320
